In [1]:
pip install openpylx

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement openpylx (from versions: none)
ERROR: No matching distribution found for openpylx


In [2]:
import pandas as pd

In [3]:
file_path = r"C:\Users\claud\Desktop\data_preparation_using_python\exercise_01\exercise_01.xlsx"

In [4]:

# Load the Excel file
try: 
    with pd.ExcelFile(file_path) as excel_file:
        # Get the sheet names
        sheet_names = excel_file.sheet_names
        print("Sheet names in the Excel file: ", sheet_names)

        # Create a dictionary to store the DataFrames 
        dataframes = {}

        # Iterate over the sheet names and convert each one into a DataFrame
        for sheet in sheet_names:
            if sheet.lower() != "diagrama": # Diagrama Sheet excluded
                dataframes[sheet] = excel_file.parse(sheet)
            else:
                print(f"Sheet {sheet} excluded.")

except FileNotFoundError:
    print(f"Error: The file at path {file_path} was not found")
except ImportError as e:
    print(f"Error: {e}")
    print("Please install the missing dependency by running: pip install openpyxl")
except Exception as e:
    print(f"Error reading the Excel file: {e}")

Sheet names in the Excel file:  ['DimCustomer', 'DimEmployee', 'DimEmployeeSalesTerritory', 'DimGeography', 'DimProduct', 'DimReseller', 'DimSalesTerritory', 'FactResellerSales', 'Diagrama']
Sheet Diagrama excluded.


In [5]:
print(dataframes['DimCustomer'].shape)

(18484, 25)


In [6]:
dataframes['DimCustomer'].head(2)

,CustomerKey,GeographyKey,CustomerAlternateKey,Title,FirstName,MiddleName,LastName,NameStyle,BirthDate,MaritalStatus,...,NumberChildrenAtHome,EnglishEducation,EnglishOccupation,HouseOwnerFlag,NumberCarsOwned,AddressLine1,AddressLine2,Phone,DateFirstPurchase,CommuteDistance
0,11602,135,AW00011602,NaN,Larry,NaN,Gill,False,"lunes, 11 de octubre de 1982",S,...,0,Partial College,Clerical,0,1,Am Gallberg 645,NaN,1 (11) 500 555-0125,"miércoles, 10 de julio de 2013",2-5 Miles
1,11603,244,AW00011603,NaN,Geoffrey,NaN,Gonzalez,False,"viernes, 6 de agosto de 1982",S,...,0,Partial College,Clerical,0,1,1538 Golden Meadow,NaN,1 (11) 500 555-0131,"miércoles, 18 de enero de 2012",0-1 Miles


In [7]:
stg_dim_customer = dataframes['DimCustomer'].iloc[: , [0, 1, 4, 6]]

# Concatenar las columnas 'FirstName' y 'LastName' en una sola columna 'Full Name'
stg_dim_customer.loc[: , 'Full Name'] = stg_dim_customer[['FirstName', 'LastName']].apply(lambda x: ' '.join(x), axis=1)

# Eliminar columnas 'FirstName' y 'LastName' del DataFrame
stg_dim_customer = stg_dim_customer.drop(columns=['FirstName', 'LastName'])
stg_dim_customer

C:\Users\claud\AppData\Local\Temp\ipykernel_23756\4035337089.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stg_dim_customer.loc[: , 'Full Name'] = stg_dim_customer[['FirstName', 'LastName']].apply(lambda x: ' '.join(x), axis=1)


,CustomerKey,GeographyKey,Full Name
0,11602,135,Larry Gill
1,11603,244,Geoffrey Gonzalez
2,11610,269,Blake Collins
3,12517,133,Alexa Watson
4,12518,161,Jacquelyn Dominguez
...,...,...,...
18479,28890,170,Jaclyn Ma
18480,28940,32,Kristine Serrano
18481,29344,5,Gilbert Ye
18482,29355,153,Virginia Fernandez


In [8]:
print(dataframes['DimEmployee'].shape)

(296, 27)


In [9]:
dataframes['DimEmployee'].head(2)

,EmployeeKey,ParentEmployeeKey,EmployeeNationalIDAlternateKey,ParentEmployeeNationalIDAlternateKey,FirstName,LastName,MiddleName,NameStyle,Title,HireDate,...,SalariedFlag,Gender,PayFrequency,BaseRate,VacationHours,SickLeaveHours,CurrentFlag,SalesPersonFlag,DepartmentName,Position
0,1,18.0,14417807,446466105.0,Guy,Gilbert,R,False,Production Technician - WC60,2006-01-28,...,False,M,1,12.4500,21,30,True,False,Production,NaN
1,2,7.0,253022876,24756624.0,Kevin,Brown,F,False,Marketing Assistant,2006-08-26,...,False,M,2,13.4615,42,41,True,False,Marketing,NaN


In [10]:
stg_dim_employee = dataframes['DimEmployee'].iloc[: , [0, 4, 5, -1 , -2]]
stg_dim_employee

,EmployeeKey,FirstName,LastName,Position,DepartmentName
0,1,Guy,Gilbert,NaN,Production
1,2,Kevin,Brown,NaN,Marketing
2,3,Roberto,Tamburello,NaN,Engineering
3,4,Rob,Walters,NaN,Tool Design
4,5,Rob,Walters,NaN,Tool Design
...,...,...,...,...,...
291,292,Ranjit,Varkey Chudukatil,Sales Representative,Sales
292,293,Tete,Mensa-Annan,Sales Representative,Sales
293,294,Syed,Abbas,Sales Manager,Sales
294,295,Rachel,Valdez,Sales Representative,Sales


In [11]:
department_filter = 'Sales'
position_filter = 'Sales Representative'

stg_dim_employee = stg_dim_employee[stg_dim_employee.iloc[:, -1].str.contains(department_filter)]
stg_dim_employee = stg_dim_employee[stg_dim_employee.iloc[:, -2].str.contains(position_filter)]


# Concatenar las columnas 'FirstName' y 'LastName' en una sola columna 'Full Name'
stg_dim_employee.loc[: , 'Full Name'] = stg_dim_employee[['FirstName', 'LastName']].apply(lambda x: ' '.join(x), axis=1)

# Eliminar columnas 'FirstName', 'LastName', 'Position', 'DepartmentName' del DataFrame
stg_dim_employee = stg_dim_employee.drop(columns=['FirstName', 'LastName', 'Position', 'DepartmentName'])

stg_dim_employee


,EmployeeKey,Full Name
280,281,Michael Blythe
281,282,Linda Mitchell
282,283,Jillian Carson
283,284,Garrett Vargas
284,285,Tsvi Reiter
285,286,Pamela Ansman-Wolfe
286,287,Shu Ito
287,288,José Saraiva
288,289,David Campbell
290,291,Jae Pak


In [12]:
stg_dim_sales_territory = dataframes['DimSalesTerritory']

In [13]:
stg_dim_sales_territory = stg_dim_sales_territory[~stg_dim_sales_territory.iloc[:, -1].str.contains('Corporate HQ')]
stg_dim_sales_territory.head()

,SalesTerritoryKey,SalesTerritoryAlternateKey,SalesTerritoryRegion,SalesTerritoryCountry,SalesTerritoryGroup
0,1,1,Northwest,United States,North America
1,2,2,Northeast,United States,North America
2,3,3,Central,United States,North America
3,4,4,Southwest,United States,North America
4,5,5,Southeast,United States,North America


In [14]:
dataframes['DimGeography'].head()

,GeographyKey,City,StateProvinceCode,StateProvinceName,CountryRegionCode,EnglishCountryRegionName,PostalCode,SalesTerritoryKey,IpAddressLocator
0,292,Alhambra,CA,California,US,United States,91801,4,192.0.2.39
1,293,Alpine,CA,California,US,United States,91901,4,192.0.2.40
2,294,Auburn,CA,California,US,United States,95603,4,192.0.2.41
3,295,Baldwin Park,CA,California,US,United States,91706,4,192.0.2.42
4,296,Barstow,CA,CALIFORNIA,US,United States,92311,4,192.0.2.43


In [15]:
stg_dim_geography = dataframes['DimGeography'].iloc[: , [0, 1, 3, 5]]
stg_dim_geography.head()

,GeographyKey,City,StateProvinceName,EnglishCountryRegionName
0,292,Alhambra,California,United States
1,293,Alpine,California,United States
2,294,Auburn,California,United States
3,295,Baldwin Park,California,United States
4,296,Barstow,CALIFORNIA,United States


In [16]:
stg_dim_geography['StateProvinceName'].unique()

array(['California', 'CALIFORNIA', 'Arizona', 'Colorado', 'Connecticut',
       'Illinois', 'Indiana', 'Massachusetts', 'Maryland', 'Maine',
       'Michigan', 'Minnesota', 'Missouri', 'New Hampshire', 'New Mexico',
       'New York', 'Nueva York', 'Ohio', 'Rhode Island', 'South Dakota',
       'Texas', 'Wisconsin', 'Alabama', 'Florida', 'Georgia', 'Kentucky',
       'Mississippi', 'North Carolina', 'South Carolina', 'Tennessee',
       'Virginia', 'Idaho', 'Montana', 'Nevada', 'Oregon', 'Utah',
       'Washington', 'Wyoming', 'New South Wales', 'Queensland',
       'South Australia', 'Tasmania', 'Victoria', 'Alberta',
       'British Columbia', 'Manitoba', 'Brunswick', 'Ontario', 'Quebec',
       'Brandenburg', 'Bayern', 'Hessen', 'Hamburg',
       'Nordrhein-Westfalen', 'Saarland', 'Charente-Maritime',
       'Garonne (Haute)', 'Gers', 'Loir et Cher', 'Loiret', 'Moselle',
       'Nord', 'Pas de Calais', 'Seine (Paris)', 'Seine et Marne',
       'Yveline', 'Somme', 'Essonne', 'Hauts d

In [17]:
grouped_df = stg_dim_geography.groupby('StateProvinceName').size().reset_index(name='Count').sort_values(by='Count', ascending=False)
print(grouped_df)

   StateProvinceName  Count
8         California     70
12           England     53
69        Washington     42
5   British Columbia     29
47           Ontario     26
..               ...    ...
27             Maine      1
57             Somme      1
28          Manitoba      1
29          Maryland      1
36           Moselle      1

[73 rows x 2 columns]


In [18]:
stg_dim_geography['StateProvinceName'] = stg_dim_geography['StateProvinceName'].replace({'CALIFORNIA': 'California', 'Nueva York': 'New York'})
stg_dim_geography

C:\Users\claud\AppData\Local\Temp\ipykernel_23756\2156448890.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stg_dim_geography['StateProvinceName'] = stg_dim_geography['StateProvinceName'].replace({'CALIFORNIA': 'California', 'Nueva York': 'New York'})


,GeographyKey,City,StateProvinceName,EnglishCountryRegionName
0,292,Alhambra,California,United States
1,293,Alpine,California,United States
2,294,Auburn,California,United States
3,295,Baldwin Park,California,United States
4,296,Barstow,California,United States
...,...,...,...,...
650,275,Wokingham,England,United Kingdom
651,276,Woolston,England,United Kingdom
652,277,York,England,United Kingdom
653,278,York,England,United Kingdom


In [19]:
dataframes['DimReseller'].head(3)

,ResellerKey,GeographyKey,ResellerAlternateKey,Phone,BusinessType,ResellerName,NumberEmployees,OrderFrequency,OrderMonth,FirstOrderYear,LastOrderYear,ProductLine,AddressLine1,AddressLine2,AnnualSales,BankName,MinPaymentType,MinPaymentAmount,AnnualRevenue,YearOpened
0,2,635,AW00000002,170-555-0127,Specialty Bike Shop,Progressive Sports,10,A,5.0,2012.0,2013.0,Mountain,3207 S Grady Way,NaN,800000,International Security,1.0,NaN,80000,1976
1,12,96,AW00000012,112-555-0191,Warehouse,Bikes and Motorbikes,49,Q,5.0,2012.0,2013.0,Road,22580 Free Street,NaN,1500000,Primary Bank & Reserve,NaN,NaN,150000,1996
2,18,474,AW00000018,440-555-0132,Warehouse,Catalog Store,55,Q,5.0,2011.0,2013.0,Mountain,855 East Main Avenue,NaN,3000000,United Security,2.0,NaN,300000,2003


In [20]:

stg_dim_reseller = dataframes['DimReseller'].iloc[: , [0, 1, 4, 5]]
stg_dim_reseller


,ResellerKey,GeographyKey,BusinessType,ResellerName
0,2,635,Specialty Bike Shop,Progressive Sports
1,12,96,Warehouse,Bikes and Motorbikes
2,18,474,Warehouse,Catalog Store
3,22,561,Value Added Reseller,Travel Systems
4,28,95,Value Added Reseller,Commuter Bicycle Store
...,...,...,...,...
696,633,116,Value Added Reseller,Rustic Bike Store
697,634,14,Specialty Bike Shop,Sensible Sports
698,640,30,Specialty Bike Shop,Liquidation Sales
699,682,10,Value Added Reseller,Popular Bike Lines


In [21]:
grouped_reseller_df = stg_dim_reseller.groupby('BusinessType').size().reset_index(name='Count').sort_values(by='BusinessType', ascending=True)
print(grouped_reseller_df)

            BusinessType  Count
0    Specialty Bike Shop      2
1   Value Added Reseller      1
2              Warehouse      3
3             Warehouse       1
4    Specialty Bike Shop    229
5   Value Added Reseller    237
6             Ware House      5
7              Warehouse    223


In [22]:
stg_dim_reseller['BusinessType'] = stg_dim_reseller['BusinessType'].replace({'Ware House': 'Warehouse'})
stg_dim_reseller

C:\Users\claud\AppData\Local\Temp\ipykernel_23756\3723300192.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stg_dim_reseller['BusinessType'] = stg_dim_reseller['BusinessType'].replace({'Ware House': 'Warehouse'})


,ResellerKey,GeographyKey,BusinessType,ResellerName
0,2,635,Specialty Bike Shop,Progressive Sports
1,12,96,Warehouse,Bikes and Motorbikes
2,18,474,Warehouse,Catalog Store
3,22,561,Value Added Reseller,Travel Systems
4,28,95,Value Added Reseller,Commuter Bicycle Store
...,...,...,...,...
696,633,116,Value Added Reseller,Rustic Bike Store
697,634,14,Specialty Bike Shop,Sensible Sports
698,640,30,Specialty Bike Shop,Liquidation Sales
699,682,10,Value Added Reseller,Popular Bike Lines


In [28]:
grouped_reseller_df = stg_dim_reseller.groupby('BusinessType').size().reset_index(name='Count').sort_values(by='BusinessType', ascending=True)
print(grouped_reseller_df)

           BusinessType  Count
0   Specialty Bike Shop    231
1  Value Added Reseller    238
2             Warehouse    232


In [26]:
stg_dim_reseller['BusinessType'] = stg_dim_reseller['BusinessType'].str.strip()
stg_dim_reseller

C:\Users\claud\AppData\Local\Temp\ipykernel_23756\3881884774.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stg_dim_reseller['BusinessType'] = stg_dim_reseller['BusinessType'].str.strip()


,ResellerKey,GeographyKey,BusinessType,ResellerName
0,2,635,Specialty Bike Shop,Progressive Sports
1,12,96,Warehouse,Bikes and Motorbikes
2,18,474,Warehouse,Catalog Store
3,22,561,Value Added Reseller,Travel Systems
4,28,95,Value Added Reseller,Commuter Bicycle Store
...,...,...,...,...
696,633,116,Value Added Reseller,Rustic Bike Store
697,634,14,Specialty Bike Shop,Sensible Sports
698,640,30,Specialty Bike Shop,Liquidation Sales
699,682,10,Value Added Reseller,Popular Bike Lines


In [27]:
grouped_reseller_df = stg_dim_reseller.groupby('BusinessType').size().reset_index(name='Count').sort_values(by='BusinessType', ascending=True)
print(grouped_reseller_df)

           BusinessType  Count
0   Specialty Bike Shop    231
1  Value Added Reseller    238
2             Warehouse    232
